In [1]:
## import the tools
import torch
import glob
from pathlib import Path
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors as NN
import glob
import pandas as pd
import ipdb
import laspy

In [2]:
## import the model tools
from torch_geometric.transforms import Compose
from torch_points3d.core.data_transform import GridSampling3D, MinPoints, XYZFeature, AddFeatsByKeys
from torch_points3d.applications.pretrained_api import PretainedRegistry
from torch_geometric.data import Batch,Dataset, Data ,DataLoader

In [3]:
## Load Powerline Model and change config
pl_models_path = "SEUNet18.pt"
model = torch.load(pl_models_path)
model['run_config']['data']['dataroot'] = '/home/frederik/data/'
model['run_config']['data']['processed_folder'] = 'processed'
torch.save(model, "SEUNet18_modified.pt")
model_pl = PretainedRegistry.from_file("SEUNet18_modified").cuda()

## transformer
pos_z = [ "pos_z" ]
list_add_to_x = [ True ]
delete_feats = [ True ]
lparams = ['512']
first_subsampling = model['run_config']['data']['first_subsampling']

transform_test = Compose([MinPoints(512),
                     XYZFeature(add_x=False, add_y=False, add_z= True),
                     AddFeatsByKeys(list_add_to_x=list_add_to_x, feat_names= pos_z,delete_feats=delete_feats),
                     GridSampling3D(mode='last', size=first_subsampling, quantize_coords=True)])

In [4]:
## to find the neighbor points prediction
from sklearn.neighbors import BallTree, KDTree
import numpy as np
def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""
    # src : whole gt points in a file 
    tree = KDTree(candidates, leaf_size=20, metric='euclidean')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    closest = np.squeeze(indices)
    closest_dist = distances
    return closest

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
loading processed train split
Total of 135 samples in train set.
loading processed val split
Total of 59 samples in val set.
loading processed test split
Total of 154 samples in test set.


In [6]:
#model_pl = PretainedRegistry.from_file("SEUNet18_modified").cuda()
#PretainedRegistry.from_pretrained(model_tag, download=True, out_file=None, weight_name="latest", mock_dataset=True)

## Process with the normalize data

In [13]:
## load test pt with normalized

## load transform pt pre
# root_path = "/home/dmn774/data/Deep1/SegmentationDenmark/Lidar_tiles_KU/Powerlines/denmark/processed/"
# root_path = "/home/dmn774/data/Deep1/SegmentationDenmark/Lidar_tiles_KU/Powerlines/test_some/denmark/processed/"

process_file_name_path = test_fold + "/processed_file_names.pt"
vis_out_folder = data_root_path + 'vis/'

pre_transform = torch.load(pre_trans_path)
process_file_name = torch.load(process_file_name_path)

In [14]:
print(pre_transform)

{'room_names': ['PUNKTSKY_00005_1km_6162_472', 'PUNKTSKY_00005_1km_6161_465'], 'room_coord_min': [array([4.7200001e+05, 6.1620000e+06, 2.3660000e+01]), array([4.6500001e+05, 6.1610000e+06, 2.3660000e+01])], 'room_coord_max': [array([4.72999920e+05, 6.16299999e+06, 1.20160000e+02]), array([4.65999990e+05, 6.16199999e+06, 1.20160000e+02])], 'room_coord_scale': [array([99.991, 99.999, 96.5  ]), array([99.998, 99.999, 96.5  ])], 'global_z': (23.66, 120.16)}


In [15]:
## loop for every files
## delete files
count = 0
global_z = pre_transform['global_z']
# for i in range(count_list):
room_names = pre_transform['room_names']
room_coord_mins = pre_transform['room_coord_min']
room_coord_maxs = pre_transform['room_coord_max']
room_coord_scales = pre_transform['room_coord_scale']
count = 0
for file in glob.glob(test_fold + "*_cloud*pt"):
#     print(file)
    sample = os.path.join(test_fold, file)
    pt_data = torch.load(sample)
    room_index = pt_data['room_idx']
    room_name = room_names[room_index]
    vis_out = os.path.join(vis_out_folder, room_name)
#     ipdb.set_trace()

    room_coord_scale = room_coord_scales[room_index]
    pos_ = pt_data['points']
    pt_ori = pt_data['points'] * room_coord_scale + pt_data['coord_min']
    
    data_s = transform_test(Batch(pos=torch.from_numpy(pos_).float(), batch=torch.zeros(pos_.shape[0]).long()))
    # data_s = transform_test(Batch(pos=torch.from_numpy(pos_).float(), batch=torch.zeros(1).long()))
    data_s.y = torch.zeros(data_s.batch.shape).long()
    f = get_nearest(pos_, data_s.pos)

    with torch.no_grad():
        model_pl.set_input(data_s, "cuda")
        model_pl.forward(data_s)
    
    pre = model_pl.output.cpu().numpy()
    m= torch.nn.functional.softmax(torch.tensor(pre), dim=1)
    cla_pre = np.argmax(m, axis=1)
    pre_ori = np.arange(len(pos_))
    for i in pre_ori:
#         print(i)
        pre_ori[i] = cla_pre[f[i]]
    combine_pre = np.column_stack((pt_ori, pre_ori.T))
#     import ipdb;ipdb.set_trace()
    vis_out = vis_out_folder + room_name +'pre.txt'
    if os.path.exists(vis_out):
        file_save = open(vis_out, 'a')
    else:
        file_save = open(vis_out, 'w')

    file_save = open(vis_out_folder + room_name +'pre.txt', 'a')
    np.savetxt(file_save, combine_pre, fmt = '%1.5f')
#     file_save.write("\n")    
#     break
print("save finished")

AttributeError: 'int' object has no attribute 'unsqueeze'

### Simple vis

In [ ]:
## read path
vis_out_folder
pred_path = os.path.join(vis_out_folder, "PUNKTSKY_00005_1km_6084_519pre.txt")
pred_data = pd.read_csv(pred_path, sep=" ", header=None).values
pred_data, pred_data.shape

In [ ]:
idx = np.arange(len(pred_data))
np.random.shuffle(idx)
idx = idx[:30000]

vmin = pred_data[idx, -1].min()
vmax = pred_data[idx, -1].max()
cm = plt.cm.get_cmap('RdYlBu')

In [ ]:
## vis
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
plt.figure()
sc = plt.scatter(pred_data[idx, 0], pred_data[idx, 1], c =pred_data[idx, -1],
                cmap=cm, vmin=vmin, vmax=vmax, s=5
            )
plt.colorbar(sc)

### check the individual pt


In [251]:
### check the individual pt
data_root_path = "/home/frederik/data/denmark/processed/"
test_fold = data_root_path + "test_0_(0.1, 0.1)/"
pre_trans_path = test_fold + "/stats.pt"
pre_transform = torch.load(pre_trans_path)

# Define global variables
global_z = pre_transform['global_z']
room_names = pre_transform['room_names']
room_coord_mins = pre_transform['room_coord_min']
room_coord_maxs = pre_transform['room_coord_max']
room_coord_scales = pre_transform['room_coord_scale']

# Load 1 Sample
onesample = r"/home/frederik/data/denmark/processed/test_0_(0.1, 0.1)/PUNKTSKY_00005_1km_6161_465_cloud_50.pt"
pt_data = torch.load(onesample)

# Define Local Variables
points = pt_data['points']
room_index = pt_data['room_idx']
room_coord_min = pt_data['coord_min']
room_coord_scale = room_coord_scales[room_index]
room_name = room_names[room_index]
pt_ori = points * room_coord_scale + room_coord_min

# Transform data corresponding to the 
data_s = transform_test(Batch(pos=torch.from_numpy(points).float()))
data_s.batch = torch.zeros(len(data_s.pos))
data_s.y = torch.zeros(data_s.pos.shape[0]).long()

# Apply Model
with torch.no_grad():
    model_pl.set_input(data_s, "cuda")
    model_pl.forward(data_s)

# Use scores
scores = model_pl.output.cpu().numpy()
probabilities = torch.nn.functional.softmax(torch.tensor(scores), dim=1)
predictions = np.argmax(probabilities, axis=1)

# Map indexes from x,y,z to the voxelized data and their predictions
indexes = get_nearest(points, data_s.pos)

points_predictions = torch.tensor([predictions[index] for _, index in enumerate(indexes)])


In [7]:
# points = np.array([[5.00069739, 2.00061773, 0.50123249]])

# data_s = transform_test(Batch(pos=torch.from_numpy(points).float()))
# data_s.batch = torch.zeros(len(data_s.pos))
# data_s.y = torch.zeros(data_s.pos.shape[0]).long()


# # Apply Model
# with torch.no_grad():
#     model_pl.set_input(data_s, "cuda")
#     model_pl.forward(data_s)

#f = get_nearest(points, data_s.pos)



#print(f)
#print(type(f))
#print((f.size))
#print(len(f))

In [68]:
# Load 1 Sample

model_pl.eval()
points_list = []
preds_list = []

### check the individual pt
data_root_path = "/home/frederik/data/denmark/processed/"
test_fold = data_root_path + "test_0_(0.1, 0.1)/"
pre_trans_path = test_fold + "/stats.pt"
pre_transform = torch.load(pre_trans_path)

# Define global variables
global_z = pre_transform['global_z']
room_names = pre_transform['room_names']
room_coord_mins = pre_transform['room_coord_min']
room_coord_maxs = pre_transform['room_coord_max']
room_coord_scales = pre_transform['room_coord_scale']


for i in range(79, 154):
    onesample = r"/home/frederik/data/denmark/processed/test_0_(0.1, 0.1)/"+"PUNKTSKY_00005_1km_6161_465_"+str(i)+"_cloud_"+str(i)+".pt"
    pt_data = torch.load(onesample)
    
    # Define Local Variables
    points = pt_data['points']
    room_index = pt_data['room_idx']
    room_coord_min = pt_data['coord_min']
    room_coord_max = pt_data['coord_max']
    
    room_coord_scale = room_coord_scales[room_index]
    room_name = room_names[room_index]
    
    pt_ori = points * (room_coord_scale + room_coord_min)
    
    # Transform data corresponding to the 
    data_s = transform_test(Batch(pos=torch.from_numpy(points).float()))
    data_s.batch = torch.zeros(len(data_s.pos))
    data_s.y = torch.zeros(data_s.pos.shape[0]).long()
    
    # Apply Model
    with torch.no_grad():
        model_pl.set_input(data_s, "cuda")
        model_pl.forward(data_s)

    # Use scores
    scores = model_pl.output.cpu().numpy()
    probabilities = torch.nn.functional.softmax(torch.tensor(scores), dim=1)
    predictions = np.argmax(probabilities, axis=1)
    
    # Map indexes from x,y,z to the voxelized data and their predictions
    indexes = get_nearest(points, data_s.pos)
    
    points_predictions = [predictions[index].numpy() for _, index in enumerate(indexes)]
    
    preds_list.append(points_predictions)
    points_list.append(pt_ori)
    
preds_list = np.array([item for sublist in preds_list for item in sublist])
points_list = np.array([item for sublist in points_list for item in sublist])


    

4095


In [ ]:
indexes = get_nearest(points_list, points_data)

In [69]:
points_list[:10].astype(int)

array([[  10278, 5341727,     -21],
       [   9767, 5344807,     -19],
       [   8930, 5340494,     -21],
       [   7627, 5339878,     -21],
       [   7162, 5342959,     -19],
       [   6185, 5338646,     -21],
       [   4930, 5337414,     -21],
       [   3581, 5336182,     -21],
       [   2186, 5334949,     -21],
       [   1488, 5336798,     -20]])

In [72]:
las = laspy.read("/home/frederik/data/denmark/raw/test/LazFilesWithHeightParam/PUNKTSKY_00005_1km_6162_472_hag_nn.laz", laz_backend=laspy.compression.LazBackend.LazrsParallel)
points_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))
points_data[:10].astype(int)

4095


array([[ 47204681, 616295176,      2529],
       [ 47204653, 616295183,      2531],
       [ 47204619, 616295192,      2529],
       [ 47204591, 616295200,      2529],
       [ 47204561, 616295208,      2529],
       [ 47204528, 616295216,      2530],
       [ 47204499, 616295224,      2535],
       [ 47204465, 616295233,      2528],
       [ 47204434, 616295241,      2532],
       [ 47204404, 616295249,      2529]])

In [61]:
points, data_s.pos

(array([[ 9.08228165,  9.14299143, -0.11792746],
        [ 9.0850817 ,  9.14309143, -0.12196891],
        [ 9.08778176,  9.14279143, -0.1211399 ],
        ...,
        [ 9.71859437,  9.83229832, -0.08165803],
        [ 9.72169443,  9.83449834, -0.11243523],
        [ 9.7247945 ,  9.83669837, -0.14373057]]),
 tensor([[ 9.5332,  9.8011, -0.1501],
         [ 9.5105,  9.8232, -0.1501],
         [ 9.5416,  9.8213, -0.1502],
         ...,
         [ 9.6701,  9.2714,  0.0138],
         [ 9.6379,  9.1803,  0.0309],
         [ 9.6833,  9.1938,  0.0338]]))

In [34]:
new_las = laspy.read("/home/frederik/data/denmark/raw/test/NewLaz/PUNKTSKY_00005_1km_6162_472.laz", laz_backend=laspy.compression.LazBackend.LazrsParallel)
new_las.X

4095


array([47230767, 47230733, 47230698, ..., 47261096, 47261067, 47261169],
      dtype=int32)